<img src="python_ecosystem.png">

(image: Ondřej Čertík)

In [1]:
import numpy as np

In [2]:
lst = list(range(10))

a = np.asarray(lst)

a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Arithmetic operations with arrays typically work elementwise

In [3]:
a + 1

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [4]:
a * 2

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

... which differs from Python lists and other sequences!

In [5]:
lst * 2

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
a**2

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

##  Boolean arrays

In [5]:
a**2 < 42

array([ True,  True,  True,  True,  True,  True,  True, False, False, False], dtype=bool)

In [6]:
lst < 42

TypeError: unorderable types: list() < int()

If for you the result of `lst < 42` is `False`, it means you are running Python 2, where a non-empty sequence is *"truthy"*.

** Boolean arrays can be used as indexing masks: **

In [7]:
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [8]:
a[a**2 < 42]

array([0, 1, 2, 3, 4, 5, 6])

In [11]:
10 < a**2 < 42

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [14]:
(10 < a**2) & (a**2 < 42)

array([False, False, False, False,  True,  True,  True, False, False, False], dtype=bool)

Note the need to use the bitwise-AND operator and brackets.

Alternatively, use the special numpy syntax:

In [10]:
np.logical_and(10 < a**2, a**2 < 42)

array([False, False, False, False,  True,  True,  True, False, False, False], dtype=bool)

**Caveat**: Assignments with boolean indexing modify the array in-place

In [11]:
a[(10 < a**2) & (a**2 < 42)] = -42

In [12]:
a

array([  0,   1,   2,   3, -42, -42, -42,   7,   8,   9])

More info: read up on *fancy indexing* vs *basic indexing*

## Arrays have useful methods

In [13]:
len(dir(a))

158

In [14]:
am = (a - a.mean()) / a.std()

In [15]:
# compare to
from __future__ import division

a_mean = sum(x for x in a) / len(a)

from math import sqrt
a_std = sqrt(sum((x - a_mean)**2 for x in a) / len(a))

am_list = [(x - a_mean) / a_std for x in a]

In [16]:
am_list - am

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [17]:
am.median()

AttributeError: 'numpy.ndarray' object has no attribute 'median'

In [18]:
np.median(a)

1.5

## Arrays can be reshaped in $O(1)$ in time and memory

In [19]:
a.shape

(10,)

In [20]:
a.reshape(2, -1)

array([[  0,   1,   2,   3, -42],
       [-42, -42,   7,   8,   9]])

In [21]:
b = a.reshape(-1, 2)

In [26]:
b

array([[  0,   1],
       [  2,   3],
       [-42, -42],
       [-42,   7],
       [  8,   9]])

In [22]:
a

array([  0,   1,   2,   3, -42, -42, -42,   7,   8,   9])

## Elementwise operations can be applied along an axis

In [23]:
np.mean(b, axis=1)

array([  0.5,   2.5, -42. , -17.5,   8.5])

In [24]:
np.mean(b, axis=1, keepdims=True)

array([[  0.5],
       [  2.5],
       [-42. ],
       [-17.5],
       [  8.5]])

## Slicing

A worked example: Neighborhood average of a two-dimensional array

From a student's email:

> A general issue of speed for the overall program. A single run with sufficient data points is taking about 2-3 weeks.


In [26]:
m, n = 4, 4
a = np.arange(m*n, dtype=float).reshape((m, n))
a

array([[  0.,   1.,   2.,   3.],
       [  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.],
       [ 12.,  13.,  14.,  15.]])

In [27]:
# a non-vectorized way

b = np.zeros((m-1, n-1))
for i in range(m-1):
    for j in range(n-1):
        b[i, j] = a[i, j] + a[i+1, j] + a[i, j+1] + a[i+1, j+1]
b

array([[ 10.,  14.,  18.],
       [ 26.,  30.,  34.],
       [ 42.,  46.,  50.]])

In [28]:
# the syntax for a slice is `start:stop:step`

a[1:3, 0]

array([ 4.,  8.])

In [29]:
a[1:-1, ...]

array([[  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.]])

In [30]:
a[1:, ...]

array([[  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.],
       [ 12.,  13.,  14.,  15.]])

In [32]:
# a vectorized expression

b_vect = a[:-1, :-1] + a[1:, :-1] + a[:-1, 1:] + a[1:, 1:]

In [33]:
np.all(b_vect == b)

True

In [34]:
N = 1000
np.random.seed(1234)
r = np.random.random((N, N))

In [35]:
%%timeit 

r_av = np.zeros((N-1, N-1))
for i in range(N-1):
    for j in range(N-1):
        r_av[i, j] = r[i, j] + r[i+1, j] + r[i, j+1] + r[i+1, j+1]

1 loop, best of 3: 1.31 s per loop


In [36]:
%%timeit 

r_av = r[:-1, :-1] + r[1:, :-1] + r[:-1, 1:] + r[1:, 1:]

100 loops, best of 3: 8.43 ms per loop


In [38]:
1.31 / 8.43e-3

155.3973902728351

#### Conway's game of life

Cells live on a square grid. Each cell can be in either of two states: alive or dead. Cells interact with nearest neighbors.

At each *tick*, 

- Any live cell with `<2` neighbors dies, as if of underpopulation.
- Any live cell with `>3` neighbors dies, as if of overpopulation.
- Any dead cell with `=3` neighbors becomes a live cell, as if by reproduction.

From a cell-centric view to a whole-array formulation: for each cell, consider the sum of nine fields.
- If the sum `= 3`, central cenral cell's state is life.
- If the sum `= 4`, the state of the central cell does not change
- Otherwise, it dies.

In [39]:
def step(X):
    """Given a game board ``X``, make a time step and return the result.
    
    NB: In this implementation the game field is finite.

    """
    num_neighb = (X[:-2, :-2]  + X[1:-1, :-2]  + X[2:, :-2] +
                  X[:-2, 1:-1] + X[1:-1, 1:-1] + X[2:, 1:-1] +
                  X[:-2, 2:]   + X[1:-1, 2:]   + X[2:, 2:])
    
    X[1:-1, 1:-1][num_neighb == 3] = 1
    X[1:-1, 1:-1][(num_neighb != 4) & (num_neighb != 3)] = 0
    return X

https://jakevdp.github.io/blog/2013/08/07/conways-game-of-life

In [40]:
from scipy.signal import convolve2d

window = np.ones((3, 3))

def step_alternative(X):
    nbrs_count = convolve2d(X, window, mode='same', boundary='wrap') - X
    return (nbrs_count == 3) | (X & (nbrs_count == 2))

## Broadcasting

Overheard on the numpy-discussion mailing list at some point:

OP:

> I personally think that silent Broadcasting is not a good thing. I had recently a lot of trouble with row and column vectors which got bradcastet toghether ...

Chuck Harris (numpy RM):

> It's how numpy works. 

In [41]:
a = np.arange(5)
a

array([0, 1, 2, 3, 4])

In [42]:
a[:, None]

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [44]:
a[:, None].shape

(5, 1)

In [43]:
a + a[:, None]

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 5],
       [2, 3, 4, 5, 6],
       [3, 4, 5, 6, 7],
       [4, 5, 6, 7, 8]])

Given two arrays, `S` and `P`, with shapes

$$
    \begin{aligned}
        \mathrm{S.shape} &= (\cdots, s_3, s_2, s_1) \\
        \mathrm{P.shape} &= (\cdots, p_3, p_2, p_1)
    \end{aligned}
$$

broadcasting works from the right backwards:

* If the number of dimensions of `S` and `P` is different, left-pad the smaller shape with ones.

* If $s_j = 1$ and $p_j \neq 1$, the corresponding axis of the `S` array is treated as if it were expanded to have $p_j$ elements.

* If $s_j \neq p_j \neq 1$, it's an error.

In [45]:
a + np.ones((6, 2))

ValueError: operands could not be broadcast together with shapes (5,) (6,2) 

## Universal functions, `ufuncs`

Universal functions of a single argument receive an array and work elementwise. Binary functions broadcast their arguments against each other and work elementwise.

In [49]:
a = np.array(list(range(10)), dtype=float) * np.pi / 10

In [50]:
np.sin(a)

array([ 0.        ,  0.30901699,  0.58778525,  0.80901699,  0.95105652,
        1.        ,  0.95105652,  0.80901699,  0.58778525,  0.30901699])

In [51]:
am = (a - a.mean()) / a.std()

1 / (1. + np.exp(am))

array([ 0.8273125 ,  0.77180715,  0.70482648,  0.62766976,  0.54340985,
        0.45659015,  0.37233024,  0.29517352,  0.22819285,  0.1726875 ])

In [53]:
a = np.arange(10)

np.add(a, a)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [54]:
np.multiply(a, a)

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

**NB**: `np.multiply` is *elementwise* multiplication. 

For linear algebra operations, use `np.dot`. Or, the matrix-multiply operator `@`, (Python >= 3.5 only)

In [57]:
a = np.ones((3, 4))
a @ a.T

array([[ 4.,  4.,  4.],
       [ 4.,  4.,  4.],
       [ 4.,  4.,  4.]])

### In-place operations

In [64]:
a = np.array(list(range(10)), dtype=float)
a

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

In [65]:
# pretty up printing 

opts = np.set_printoptions(precision=3)

In [66]:
# note the use of the `out=` argument 

np.exp(a, out=a)

array([  1.000e+00,   2.718e+00,   7.389e+00,   2.009e+01,   5.460e+01,
         1.484e+02,   4.034e+02,   1.097e+03,   2.981e+03,   8.103e+03])

In [67]:
a

array([  1.000e+00,   2.718e+00,   7.389e+00,   2.009e+01,   5.460e+01,
         1.484e+02,   4.034e+02,   1.097e+03,   2.981e+03,   8.103e+03])

#### `np.add.<TAB>`

- `np.add.reduce` is `np.sum`

- `np.add.accumulate` is `np.cumsum`

- `np.add.outer` has the outer-product semantics

In [69]:
# np.add.accumulate?

#### Cauchy matrix

Given two arrays $u_i$ and $v_i$, construct

$$
A_{ij} = \frac{1}{u_i - v_j}
$$

In [70]:
u = np.arange(3)
v = np.arange(3) + 0.5

A_ = np.zeros((len(u), len(v)))
for i in range(len(u)):
    for j in range(len(v)):
        A_[i, j] = 1. / (u[i] - v[j])
A_

array([[-2.   , -0.667, -0.4  ],
       [ 2.   , -2.   , -0.667],
       [ 0.667,  2.   , -2.   ]])

In [71]:
A = 1. / np.subtract.outer(u, v)
A

array([[-2.   , -0.667, -0.4  ],
       [ 2.   , -2.   , -0.667],
       [ 0.667,  2.   , -2.   ]])

Excercise: construct the Cauchy matrix from two 1D arrays without `np.subtract.outer`, using broadcasting only.


In [72]:
# Enter your code here

## Linear algebra with [numpy.linalg](https://docs.scipy.org/doc/numpy/reference/routines.linalg.html)

In [73]:
np.linalg.det(A)

-15.170370370370369

In [74]:
rhs = np.ones(3)
np.linalg.solve(A, rhs)

array([-0.188, -0.375, -0.938])

## Random number generation with [numpy.random](https://docs.scipy.org/doc/numpy/reference/routines.random.html)

In [79]:
np.random.seed(1234)    # For reproducibility, consider seeding your generator
np.random.random(size=10)

array([ 0.192,  0.622,  0.438,  0.785,  0.78 ,  0.273,  0.276,  0.802,
        0.958,  0.876])

`np.random` provides a global stream of pseudo-random numbers. Under the hood, there is a single, global  `RandomState` object. In real code prefer using explicit `RandomState` objects

In [77]:
rndm = np.random.RandomState(1234)
rndm.uniform(size=11)

array([ 0.192,  0.622,  0.438,  0.785,  0.78 ,  0.273,  0.276,  0.802,
        0.958,  0.876,  0.358])

In [78]:
rndm.normal(loc=0, scale=8, size=11)

array([ 16.02 ,   0.087,  -6.954,  11.4  ,   1.166,  23.153,  -2.434,
         6.893,  -5.519,   1.5  ,   4.834])

## (Some) `numpy` gotchas

#### `numpy` gotchas for python users

`lst[:]` is a copy of `lst`

`arr[:]` is a *view* into `arr` (for copying use `arr.copy()`)

#### `numpy` gotchas for pandas users

In [46]:
a = np.array([np.nan, 2., 3.])

import pandas as pd
s = pd.Series(a)

In [47]:
s.sum()

5.0

In `numpy`, NaN means "invalid" not "missing".

In [48]:
a.sum()

nan

In [49]:
np.nansum(a)

5.0

#### never do `from numpy import *`

See [Exercise 26](https://github.com/rougier/numpy-100/blob/master/100%20Numpy%20exercises.md#26-what-is-the-output-of-the-following-script-) of Numpy 100 exercises.

### Further reading

Numpy docs http://docs.scipy.org/doc/numpy/reference/index.html

Jake Vanderplas' *Numpy Intro*
http://nbviewer.ipython.org/github/jakevdp/2013_fall_ASTR599/blob/master/notebooks/05_NumpyIntro.ipynb

and *Efficient Numpy* http://nbviewer.ipython.org/github/jakevdp/2013_fall_ASTR599/blob/master/notebooks/11_EfficientNumpy.ipynb

or *Loosing your loops*
https://speakerdeck.com/jakevdp/losing-your-loops-fast-numerical-computing-with-numpy-pycon-2015

Scipy lecture notes, incl Pauli Virtanen's *Advanced Numpy*
https://scipy-lectures.github.io/

Nicolas Rougier's *100 Numpy exercises*
https://github.com/rougier/numpy-100/